In [1]:
import numpy as np

First, let me start this with something simple.

Let me consider y = x*2. If I want to compute dy/dx, I know the answer should be 2. But how would an autograd system compute this?

In [110]:
# Value class

class Value:
    def __init__(self, data):
        self.data = data
        self.grad = 0
        self._op = None
        self._inputs = []
    
    def __mul__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        result = Value(self.data * other.data)
        result._op = 'mul'
        result._inputs = [self, other]
        return result
    
    def backward(self):
        self.grad = 1

        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._inputs:
                    build_topo(child)
                topo.append(v)
                
        build_topo(self)

        for v in reversed(topo):
            if v._op == 'mul':
                a, b = v._inputs
                a.grad += v.grad * b.data
                b.grad += v.grad * a.data

In [111]:
x = Value(3)
y = x * 2
z = y * 5

In [112]:
z.backward()

In [113]:
print(y.grad)

5


In [114]:
print(x.grad)

10
